In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import shap
import xgboost as xgb
import lightgbm as lgm

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

import eli5
from eli5.sklearn import PermutationImportance

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('train_BRCpofr.csv')
test = pd.read_csv('test_koRSKBP.csv')

In [3]:
df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv
0,1,Male,Urban,Bachelor,5L-10L,1,5,5790,More than 1,A,Platinum,64308
1,2,Male,Rural,High School,5L-10L,0,8,5080,More than 1,A,Platinum,515400
2,3,Male,Urban,Bachelor,5L-10L,1,8,2599,More than 1,A,Platinum,64212
3,4,Female,Rural,High School,5L-10L,0,7,0,More than 1,A,Platinum,97920
4,5,Male,Urban,High School,More than 10L,1,6,3508,More than 1,A,Gold,59736


In [4]:
df['marital_status'].unique()

array([1, 0], dtype=int64)

In [5]:
df['qualification'].unique()

array(['Bachelor', 'High School', 'Others'], dtype=object)

In [6]:
df['income'].unique()

array(['5L-10L', 'More than 10L', '2L-5L', '<=2L'], dtype=object)

In [7]:
income = df['income']
df['income_int'] = np.where(income=='5L-10L', 75000, 
                              np.where(income=='<=2L', 15000, 
                              np.where(income=='More than 10L', 120000, 0)))

income = test['income']
test['income_int'] = np.where(income=='5L-10L', 75000, 
                              np.where(income=='<=2L', 15000, 
                              np.where(income=='More than 10L', 120000, 0)))

In [8]:
df['Age'] = df['vintage'].apply(lambda x: datetime.now().year - x)
df['Age'] = df['Age'].astype('int')

test['Age'] = test['vintage'].apply(lambda x: datetime.now().year - x)
test['Age'] = test['Age'].astype('int')

In [9]:
education_map = {'High School': 1, 'Bachelor': 2, 'Others': 3}
df['Education'] = df['qualification'].map(education_map)
df['Education'] = df['Education'].astype('int')

education_map = {'High School': 1, 'Bachelor': 2, 'Others': 3}
test['Education'] = test['qualification'].map(education_map)
test['Education'] = test['Education'].astype('int')

In [10]:
occupation_map = {'<=2L': 1, '2L-5L': 2, '5L-10L': 3, 'More than 10L': 4}
df['Occupation'] = df['income'].map(occupation_map)
df['Occupation'] = df['Occupation'].astype('int')

occupation_map = {'<=2L': 1, '2L-5L': 2, '5L-10L': 3, 'More than 10L': 4}
test['Occupation'] = test['income'].map(occupation_map)
test['Occupation'] = test['Occupation'].astype('int')

In [11]:
df['Income_velocity'] = df['income_int'] / 12
df['Income_velocity'] = df['Income_velocity'].astype('float')

test['Income_velocity'] = test['income_int'] / 12
test['Income_velocity'] = test['Income_velocity'].astype('float')

In [12]:
num_policies_int_ = df['num_policies']
df['num_policies_int'] = np.where(num_policies_int_=='More than 1', 2,
                                 np.where(num_policies_int_=='1', 1, 0))

num_policies_int_ = test['num_policies']
test['num_policies_int'] = np.where(num_policies_int_=='More than 1', 2,
                                 np.where(num_policies_int_=='1', 1, 0))

In [13]:
df['policy'].unique()

array(['A', 'C', 'B'], dtype=object)

In [14]:
df['Num_claims'] = df['claim_amount'].apply(lambda x: 1 if x > 0 else 0)
df['Num_claims'] = df['Num_claims'].astype('int')
df['Claim_amount_per_policy'] = df['claim_amount'] / df['num_policies_int']
df['Claim_amount_per_policy'] = df['Claim_amount_per_policy'].astype('float')
df['Claims_ratio'] = df['Num_claims'] / df['num_policies_int']
df['Claims_ratio'] = df['Claims_ratio'].astype('float')

test['Num_claims'] = test['claim_amount'].apply(lambda x: 1 if x > 0 else 0)
test['Num_claims'] = test['Num_claims'].astype('int')
test['Claim_amount_per_policy'] = test['claim_amount'] / df['num_policies_int']
test['Claim_amount_per_policy'] = test['Claim_amount_per_policy'].astype('float')
test['Claims_ratio'] = test['Num_claims'] / df['num_policies_int']
test['Claims_ratio'] = test['Claims_ratio'].astype('float')

In [46]:
df['Loyalty_score'] = df['vintage'].apply(lambda x: x * 0.1)
df['Loyalty_score'] = df['Loyalty_score'].astype('float')

df['Ratio_claim_amount_to_income'] = df['claim_amount'] / df['income_int']
df['Ratio_claim_amount_to_income'] = df['Ratio_claim_amount_to_income'].replace(np.inf, np.nan)
df['Ratio_claim_amount_to_income'] = df['Ratio_claim_amount_to_income'].fillna(df['Ratio_claim_amount_to_income'].mean(), axis=0)
df['Ratio_claim_amount_to_income'] = df['Ratio_claim_amount_to_income'].astype('float')

df['Ratio_num_policies_to_vintage'] = df['num_policies_int'] / df['vintage']
df['Ratio_num_policies_to_vintage'] = df['Ratio_num_policies_to_vintage'].replace(np.inf, np.nan)
df['Ratio_num_policies_to_vintage'] = df['Ratio_num_policies_to_vintage'].fillna(test['Ratio_claim_amount_to_income'].mean(), axis=0)
df['Ratio_num_policies_to_vintage'] = df['Ratio_num_policies_to_vintage'].astype('float')

df['Ratio_vintage_to_policy'] = df['vintage'] / df['policy_tenure']
df['Ratio_vintage_to_policy'] = df['Ratio_vintage_to_policy'].astype('float')

df['Frequency_gender'] = df.groupby('gender')['id'].transform('count')
df['Frequency_gender'] = df['Frequency_gender'].astype('int')

df['Frequency_area'] = df.groupby('area')['id'].transform('count')
df['Frequency_area'] = df['Frequency_area'].astype('int')

df['Frequency_qualification'] = df.groupby('qualification')['id'].transform('count')
df['Frequency_qualification'] = df['Frequency_qualification'].astype('int')

df['Frequency_income'] = df.groupby('income_int')['id'].transform('count')
df['Frequency_income'] = df['Frequency_income'].astype('int')

df['Frequency_marital_status'] = df.groupby('marital_status')['id'].transform('count')
df['Frequency_marital_status'] = df['Frequency_marital_status'].astype('int')

df['Frequency_vintage'] = df.groupby('vintage')['id'].transform('count')
df['Frequency_vintage'] = df['Frequency_vintage'].astype('int')

df['Frequency_claim_amount'] = df.groupby('claim_amount')['id'].transform('count')
df['Frequency_claim_amount'] = df['Frequency_claim_amount'].astype('int')

df['Frequency_num_policies'] = df.groupby('num_policies_int')['id'].transform('count')
df['Frequency_num_policies'] = df['Frequency_num_policies'].astype('int')

df['Frequency_type_of_policy'] = df.groupby('type_of_policy')['id'].transform('count')
df['Frequency_type_of_policy'] = df['Frequency_type_of_policy'].astype('int')

df['Frequency_cltv'] = df.groupby('cltv')['id'].transform('count')
df['Frequency_cltv'] = df['Frequency_cltv'].astype('int')

In [47]:
test['Loyalty_score'] = test['vintage'].apply(lambda x: x * 0.1)
test['Loyalty_score'] = test['Loyalty_score'].astype('float')

test['Ratio_claim_amount_to_income'] = test['claim_amount'] / test['income_int']
test['Ratio_claim_amount_to_income'] = test['Ratio_claim_amount_to_income'].replace(np.inf, np.nan)
test['Ratio_claim_amount_to_income'] = test['Ratio_claim_amount_to_income'].fillna(test['Ratio_claim_amount_to_income'].mean(), axis=0)
test['Ratio_claim_amount_to_income'] = test['Ratio_claim_amount_to_income'].astype('float')

test['Ratio_num_policies_to_vintage'] = test['num_policies_int'] / test['vintage']
test['Ratio_num_policies_to_vintage'] = test['Ratio_num_policies_to_vintage'].replace(np.inf, np.nan)
test['Ratio_num_policies_to_vintage'] = test['Ratio_num_policies_to_vintage'].fillna(test['Ratio_claim_amount_to_income'].mean(), axis=0)
test['Ratio_num_policies_to_vintage'] = test['Ratio_num_policies_to_vintage'].astype('float')

test['Ratio_vintage_to_policy'] = test['vintage'] / test['policy_tenure']
test['Ratio_vintage_to_policy'] = test['Ratio_vintage_to_policy'].astype('float')

test['Frequency_gender'] = test.groupby('gender')['id'].transform('count')
test['Frequency_gender'] = test['Frequency_gender'].astype('int')

test['Frequency_area'] = test.groupby('area')['id'].transform('count')
test['Frequency_area'] = test['Frequency_area'].astype('int')

test['Frequency_qualification'] = test.groupby('qualification')['id'].transform('count')
test['Frequency_qualification'] = test['Frequency_qualification'].astype('int')

test['Frequency_income'] = test.groupby('income_int')['id'].transform('count')
test['Frequency_income'] = test['Frequency_income'].astype('int')

test['Frequency_marital_status'] = test.groupby('marital_status')['id'].transform('count')
test['Frequency_marital_status'] = test['Frequency_marital_status'].astype('int')

test['Frequency_vintage'] = test.groupby('vintage')['id'].transform('count')
test['Frequency_vintage'] = test['Frequency_vintage'].astype('int')

test['Frequency_claim_amount'] = test.groupby('claim_amount')['id'].transform('count')
test['Frequency_claim_amount'] = test['Frequency_claim_amount'].astype('int')

test['Frequency_num_policies'] = test.groupby('num_policies_int')['id'].transform('count')
test['Frequency_num_policies'] = test['Frequency_num_policies'].astype('int')

test['Frequency_type_of_policy'] = test.groupby('type_of_policy')['id'].transform('count')
test['Frequency_type_of_policy'] = test['Frequency_type_of_policy'].astype('int')

test['Frequency_cltv'] = test.groupby('cltv')['id'].transform('count')
test['Frequency_cltv'] = test['Frequency_cltv'].astype('int')

KeyError: 'cltv'

In [48]:
df['Probability_claim_amount_given_income'] = df.groupby('income_int')['claim_amount'].transform('mean')
df['Probability_claim_amount_given_income'] = df['Probability_claim_amount_given_income'].astype('float')

df['Probability_num_policies_given_vintage'] = df.groupby('vintage')['num_policies_int'].transform('mean')
df['Probability_num_policies_given_vintage'] = df['Probability_num_policies_given_vintage'].astype('float')

df['Probability_vintage_given_policy'] = df.groupby('policy_tenure')['vintage'].transform('mean')
df['Probability_vintage_given_policy'] = df['Probability_vintage_given_policy'].astype('float')

In [49]:
test['Probability_claim_amount_given_income'] = test.groupby('income_int')['claim_amount'].transform('mean')
test['Probability_claim_amount_given_income'] = test['Probability_claim_amount_given_income'].astype('float')

test['Probability_num_policies_given_vintage'] = test.groupby('vintage')['num_policies_int'].transform('mean')
test['Probability_num_policies_given_vintage'] = test['Probability_num_policies_given_vintage'].astype('float')

test['Probability_vintage_given_policy'] = test.groupby('policy_tenure')['vintage'].transform('mean')
test['Probability_vintage_given_policy'] = test['Probability_vintage_given_policy'].astype('float')

In [19]:
df['income'] = df['income'].replace({'<=2': 'Less than or equal to 2L'})
test['income'] = test['income'].replace({'<=2': 'Less than or equal to 2L'})

In [20]:
df['age'] = df['vintage']/365.25
df['age'] = df['age'].astype('float')

df['family_size'] = np.where(df['marital_status']==1, 2, 1)
df['family_size'] = df['family_size'].astype('int')

df['no_of_claims'] = df['claim_amount'].count()
df['no_of_claims'] = df['no_of_claims'].astype('int')

test['age'] = test['vintage']/365.25
test['age'] = test['age'].astype('float')

test['family_size'] = np.where(test['marital_status']==1, 2, 1)
test['family_size'] = test['family_size'].astype('int')

test['no_of_claims'] = test['claim_amount'].count()
test['no_of_claims'] = test['no_of_claims'].astype('int')

In [21]:
num_policies = df['num_policies']
type_of_policy = df['type_of_policy']
df['spending_power'] =  np.where((num_policies=='More than 1') & (type_of_policy=='Silver'), 3,
                        np.where((num_policies=='More than 1') & (type_of_policy=='Gold'), 2,
                        np.where((num_policies=='More than 1') & (type_of_policy=='Platinum'), 1, 
                        np.where((num_policies=='1') & (type_of_policy=='Silver'), 6,
                        np.where((num_policies=='1') & (type_of_policy=='Gold'), 4,
                        np.where((num_policies=='1') & (type_of_policy=='Platinum'), 2, 0))))))

In [22]:
num_policies = test['num_policies']
type_of_policy = test['type_of_policy']
test['spending_power'] =  np.where((num_policies=='More than 1') & (type_of_policy=='Silver'), 3,
                        np.where((num_policies=='More than 1') & (type_of_policy=='Gold'), 2,
                        np.where((num_policies=='More than 1') & (type_of_policy=='Platinum'), 1, 
                        np.where((num_policies=='1') & (type_of_policy=='Silver'), 6,
                        np.where((num_policies=='1') & (type_of_policy=='Gold'), 4,
                        np.where((num_policies=='1') & (type_of_policy=='Platinum'), 2, 0))))))

In [23]:
qualification = df['qualification']
df['education_level'] = (np.where(qualification=='Other', 3,
                         np.where(qualification=='Bachelor', 2,
                         np.where(qualification=='High School', 1, 0))))

In [24]:
qualification = test['qualification']
test['education_level'] = (np.where(qualification=='Other', 3,
                         np.where(qualification=='Bachelor', 2,
                         np.where(qualification=='High School', 1, 0))))

In [25]:
income = df['income']
area = df['area']
df['spending_power'] =  np.where((area=='Urban') & (income=='5L - 10L'), 150000,
                        np.where((area=='Urban') & (income=='2L - 5L'), 350000,
                        np.where((area=='Urban') & (income=='<=2L'), 200000, 
                        np.where((area=='Urban') & (income=='More than 10L'), 1000000,
                        np.where((area=='Rural') & (income=='5L - 10L'), 100000,
                        np.where((area=='Rural') & (income=='2L - 5L'), 350000,
                        np.where((area=='Rural') & (income=='<=2L'), 200000, 
                        np.where((area=='Rural') & (income=='More than 10L'), 1000000, 0))))))))

In [26]:
income = test['income']
area = test['area']
test['spending_power'] =  np.where((area=='Urban') & (income=='5L - 10L'), 150000,
                        np.where((area=='Urban') & (income=='2L - 5L'), 350000,
                        np.where((area=='Urban') & (income=='<=2L'), 200000, 
                        np.where((area=='Urban') & (income=='More than 10L'), 1000000,
                        np.where((area=='Rural') & (income=='5L - 10L'), 100000,
                        np.where((area=='Rural') & (income=='2L - 5L'), 350000,
                        np.where((area=='Rural') & (income=='<=2L'), 200000, 
                        np.where((area=='Rural') & (income=='More than 10L'), 1000000, 0))))))))

In [27]:
policy = df['policy']
df['policy_tenure'] = np.where(policy=='A', 3, 
                                 np.where(policy=='B', 2, 
                                 np.where(policy=='C', 1, 0)))

df['avg_premium_amount'] = df['policy_tenure'].mean()

In [28]:
policy = test['policy']
test['policy_tenure'] = np.where(policy=='A', 3, 
                                 np.where(policy=='B', 2, 
                                 np.where(policy=='C', 1, 0)))

test['avg_premium_amount'] = test['policy_tenure'].mean()

In [29]:
num_policies_int_ = test['num_policies']
test['num_policies_int'] = np.where(num_policies_int_=='More than 1', 2,
                                 np.where(num_policies_int_=='1', 1, 0))

In [30]:
num_policies_int_ = df['num_policies']
df['num_policies_int'] = np.where(num_policies_int_=='More than 1', 2,
                                 np.where(num_policies_int_=='1', 1, 0))

In [31]:
df['avg_claim_per_id'] = df.groupby('id')['claim_amount'].mean()
df['avg_claim_per_id'] = df['avg_claim_per_id'].astype('float64')
df['avg_claim_per_id'] = df['avg_claim_per_id'].fillna(0, axis=0)

In [32]:
test['avg_claim_per_id'] = test.groupby('id')['claim_amount'].mean()
test['avg_claim_per_id'] = test['avg_claim_per_id'].astype('float64')
test['avg_claim_per_id'] = test['avg_claim_per_id'].fillna(0, axis=0)

In [33]:
income = df['income']
df['income_int'] = np.where(income=='5L-10L', 75000, 
                              np.where(income=='<=2L', 15000, 
                              np.where(income=='More than 10L', 120000, 0)))

In [34]:
income = test['income']
test['income_int'] = np.where(income=='5L-10L', 75000, 
                              np.where(income=='<=2L', 15000, 
                              np.where(income=='More than 10L', 120000, 0)))

In [35]:
df['claim_amt_to_income_ratio'] = df['claim_amount']/df['income_int']
df['no_of_policies_to_vintage_ratio'] = df['num_policies_int']/df['vintage']
df['premium_amt_to_income_ratio'] = df['policy_tenure']/df['income_int']
df['no_of_claims_to_vintage_ratio'] = df['claim_amount'].count()/df['vintage']
df["income_vintage"] = df["income_int"] * df["vintage"]
df["claim_num_policies"] = df["claim_amount"] * df["num_policies_int"]

In [36]:
test['claim_amt_to_income_ratio'] = test['claim_amount']/test['income_int']
test['no_of_policies_to_vintage_ratio'] = test['num_policies_int']/test['vintage']
test['premium_amt_to_income_ratio'] = test['policy_tenure']/test['income_int']
test['no_of_claims_to_vintage_ratio'] = test['claim_amount'].count()/test['vintage']
test["income_vintage"] = test["income_int"] * test["vintage"]
test["claim_num_policies"] = test["claim_amount"] * test["num_policies_int"]

In [37]:
df['claim_amt_to_income_ratio'] = df['claim_amt_to_income_ratio'].replace(np.inf, np.nan)
df['premium_amt_to_income_ratio'] = df['premium_amt_to_income_ratio'].replace(np.inf, np.nan)
df['no_of_policies_to_vintage_ratio'] = df['no_of_policies_to_vintage_ratio'].replace(np.inf, np.nan)
df['no_of_claims_to_vintage_ratio'] = df['no_of_claims_to_vintage_ratio'].replace(np.inf, np.nan)

In [38]:
test['claim_amt_to_income_ratio'] = test['claim_amt_to_income_ratio'].replace(np.inf, np.nan)
test['premium_amt_to_income_ratio'] = test['premium_amt_to_income_ratio'].replace(np.inf, np.nan)
test['no_of_policies_to_vintage_ratio'] = test['no_of_policies_to_vintage_ratio'].replace(np.inf, np.nan)
test['no_of_claims_to_vintage_ratio'] = test['no_of_claims_to_vintage_ratio'].replace(np.inf, np.nan)

In [39]:
df['claim_amt_to_income_ratio'] = df['claim_amt_to_income_ratio'].fillna(0, axis=0)
df['premium_amt_to_income_ratio'] = df['premium_amt_to_income_ratio'].fillna(0, axis=0)
df['no_of_policies_to_vintage_ratio'] = df['no_of_policies_to_vintage_ratio'].fillna(0, axis=0)
df['no_of_claims_to_vintage_ratio'] = df['no_of_claims_to_vintage_ratio'].fillna(0, axis=0)

In [40]:
test['claim_amt_to_income_ratio'] = test['claim_amt_to_income_ratio'].fillna(0, axis=0)
test['premium_amt_to_income_ratio'] = test['premium_amt_to_income_ratio'].fillna(0, axis=0)
test['no_of_policies_to_vintage_ratio'] = test['no_of_policies_to_vintage_ratio'].fillna(0, axis=0)
test['no_of_claims_to_vintage_ratio'] = test['no_of_claims_to_vintage_ratio'].fillna(0, axis=0)

In [1099]:
df.drop(['income_int', 'num_policies_int'], axis=1, inplace=True)

In [1100]:
test.drop(['income_int', 'num_policies_int'], axis=1, inplace=True)

In [41]:
cat_col = [col for col in df.columns if df[col].dtypes == 'object']
print(cat_col)

['gender', 'area', 'qualification', 'income', 'num_policies', 'policy', 'type_of_policy']


In [42]:
num_col = [col for col in df.columns if df[col].dtypes in ['int32', 'int64', 'float32', 'float64']]
print(num_col)

['id', 'marital_status', 'vintage', 'claim_amount', 'cltv', 'income_int', 'Age', 'Education', 'Occupation', 'Income_velocity', 'num_policies_int', 'Num_claims', 'Claim_amount_per_policy', 'Claims_ratio', 'Loyalty_score', 'Ratio_claim_amount_to_income', 'Ratio_num_policies_to_vintage', 'Probability_claim_amount_given_income', 'Probability_num_policies_given_vintage', 'age', 'family_size', 'no_of_claims', 'spending_power', 'education_level', 'policy_tenure', 'avg_premium_amount', 'avg_claim_per_id', 'claim_amt_to_income_ratio', 'no_of_policies_to_vintage_ratio', 'premium_amt_to_income_ratio', 'no_of_claims_to_vintage_ratio', 'income_vintage', 'claim_num_policies']


In [43]:
pd.set_option('display.max_columns', 27)
df.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv,income_int,...,family_size,no_of_claims,spending_power,education_level,policy_tenure,avg_premium_amount,avg_claim_per_id,claim_amt_to_income_ratio,no_of_policies_to_vintage_ratio,premium_amt_to_income_ratio,no_of_claims_to_vintage_ratio,income_vintage,claim_num_policies
0,1,Male,Urban,Bachelor,5L-10L,1,5,5790,More than 1,A,Platinum,64308,75000,...,2,89392,0,2,3,2.542901,0.0,0.077200,0.400000,0.000040,17878.400000,375000,11580
1,2,Male,Rural,High School,5L-10L,0,8,5080,More than 1,A,Platinum,515400,75000,...,1,89392,0,1,3,2.542901,5790.0,0.067733,0.250000,0.000040,11174.000000,600000,10160
2,3,Male,Urban,Bachelor,5L-10L,1,8,2599,More than 1,A,Platinum,64212,75000,...,2,89392,0,2,3,2.542901,5080.0,0.034653,0.250000,0.000040,11174.000000,600000,5198
3,4,Female,Rural,High School,5L-10L,0,7,0,More than 1,A,Platinum,97920,75000,...,1,89392,0,1,3,2.542901,2599.0,0.000000,0.285714,0.000040,12770.285714,525000,0
4,5,Male,Urban,High School,More than 10L,1,6,3508,More than 1,A,Gold,59736,120000,...,2,89392,1000000,1,3,2.542901,0.0,0.029233,0.333333,0.000025,14898.666667,720000,7016


In [44]:
df.isnull().sum()

id                                           0
gender                                       0
area                                         0
qualification                                0
income                                       0
marital_status                               0
vintage                                      0
claim_amount                                 0
num_policies                                 0
policy                                       0
type_of_policy                               0
cltv                                         0
income_int                                   0
Age                                          0
Education                                    0
Occupation                                   0
Income_velocity                              0
num_policies_int                             0
Num_claims                                   0
Claim_amount_per_policy                      0
Claims_ratio                                 0
Loyalty_score

In [1105]:
df['id'].describe()

count    89392.000000
mean     44696.500000
std      25805.391969
min          1.000000
25%      22348.750000
50%      44696.500000
75%      67044.250000
max      89392.000000
Name: id, dtype: float64

In [51]:
percentage_missing = (df.isnull().sum())/(df.shape[0]) * 100 
percentage_missing[percentage_missing>0]

Series([], dtype: float64)

In [730]:
df['Ratio_claim_amount_to_income'].fillna(np.mean, inplace=True)

In [50]:
df.to_csv('Updated_DataFrame_Train.csv', index=False)

In [52]:
test.to_csv('Updated_DataFrame_Test.csv', index=False)

In [383]:
cat_col = [col for col in df.columns if df[col].dtypes == 'object']
for col in cat_col:
    ohe = preprocessing.OneHotEncoder()
    df.loc[:, col] = ohe.fit_transform(df[col].values.reshape(-1, 1))

IndexError: tuple index out of range

In [ ]:
X = df.drop('cltv', axis=1)

In [ ]:
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
pd.DataFrame({'vif': vif}, index=X.columns)

In [ ]:
X = df.drop('cltv', axis=1)
y = df['cltv']
X_constant = sm.add_constant(X)

In [ ]:
lin_reg = sm.OLS(y, X_constant).fit()
lin_reg.summary()

In [475]:
X = df.drop(['cltv', 'Education', 'Occupation'], axis=1)
y = df['cltv']

numerical_features = [col for col in df.columns if df[col].dtypes != 'object']
categorical_features = [col for col in df.columns if df[col].dtypes == 'object']
preprocessor = ColumnTransformer(
    transformers=[("num", StandardScaler(), numerical_features), ("cat", OneHotEncoder(), categorical_features)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

ValueError: A given column is not a column of the dataframe

In [637]:
df = df.drop(['Education', 'Occupation'], axis=1)

In [853]:
[col for col in df.columns if df[col].dtypes == 'object']

['gender',
 'area',
 'qualification',
 'income',
 'num_policies',
 'policy',
 'type_of_policy']

In [1019]:
cat_cols = [col for col in df.columns if df[col].dtypes == 'object']
df = pd.get_dummies(data=df, columns=cat_cols)
df.head()

,id,marital_status,vintage,claim_amount,cltv,Age,Education,Occupation,Income_velocity,Num_claims,Claim_amount_per_policy,Claims_ratio,Loyalty_score,...,qualification_Others,income_2L-5L,income_5L-10L,income_<=2L,income_More than 10L,num_policies_1,num_policies_More than 1,policy_A,policy_B,policy_C,type_of_policy_Gold,type_of_policy_Platinum,type_of_policy_Silver
0,1,1,5,5790,64308,2018,2,3,6250.0,1,2895.0,0.5,0.5,...,0,0,1,0,0,0,1,1,0,0,0,1,0
1,2,0,8,5080,515400,2015,1,3,6250.0,1,2540.0,0.5,0.8,...,0,0,1,0,0,0,1,1,0,0,0,1,0
2,3,1,8,2599,64212,2015,2,3,6250.0,1,1299.5,0.5,0.8,...,0,0,1,0,0,0,1,1,0,0,0,1,0
3,4,0,7,0,97920,2016,1,3,6250.0,0,0.0,0.0,0.7,...,0,0,1,0,0,0,1,1,0,0,0,1,0
4,5,1,6,3508,59736,2017,1,4,10000.0,1,1754.0,0.5,0.6,...,0,0,0,0,1,0,1,1,0,0,1,0,0


In [1020]:
new_df.replace(np.inf, np.nan)

,id,marital_status,vintage,claim_amount,cltv,Age,Education,Occupation,Income_velocity,Years_at_current_residence,Num_claims,Claim_amount_per_policy,Claims_ratio,...,qualification_Others,income_2L-5L,income_5L-10L,income_Less than or equal to 2L,income_More than 10L,num_policies_1,num_policies_More than 1,policy_A,policy_B,policy_C,type_of_policy_Gold,type_of_policy_Platinum,type_of_policy_Silver
7,-0.999843,-1.0,-1.666667,0.375271,-0.044819,1.666667,0.0,0.0,0.00,-1.666667,0.0,0.088144,0.0,...,0,0,1,0,0,0,1,1,0,0,0,0,1
11,-0.999754,-1.0,0.333333,0.613341,-0.047190,-0.333333,1.0,-1.0,-1.25,0.333333,0.0,0.258233,0.0,...,0,1,0,0,0,0,1,0,0,1,0,0,1
12,-0.999732,0.0,0.666667,0.064534,-0.110031,-0.666667,1.0,-1.0,-1.25,0.666667,0.0,0.704378,0.5,...,0,1,0,0,0,1,0,1,0,0,0,1,0
13,-0.999709,0.0,-1.333333,0.142625,1.322978,1.333333,1.0,-1.0,-1.25,-1.333333,0.0,-0.078071,0.0,...,0,1,0,0,0,0,1,0,0,1,0,0,1
15,-0.999664,-1.0,-1.333333,0.354935,-0.210102,1.333333,0.0,-1.0,-1.25,-1.333333,0.0,1.119334,0.5,...,0,1,0,0,0,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89380,0.999754,0.0,-1.333333,1.571584,1.928859,1.333333,0.0,-1.0,-1.25,-1.333333,0.0,0.942852,0.0,...,0,1,0,0,0,0,1,1,0,0,0,0,1
89384,0.999843,-1.0,0.666667,-1.108731,2.083472,-0.666667,1.0,-1.0,-1.25,0.666667,-1.0,-0.972104,-0.5,...,0,1,0,0,0,0,1,0,1,0,0,1,0
89386,0.999888,0.0,-1.666667,0.448753,-0.132559,1.666667,0.0,0.0,0.00,-1.666667,0.0,1.253390,0.5,...,0,0,1,0,0,1,0,1,0,0,0,1,0
89388,0.999933,0.0,-1.666667,-0.052061,0.023714,1.666667,1.0,0.0,0.00,-1.666667,0.0,-0.217164,0.0,...,0,0,1,0,0,0,1,1,0,0,0,1,0


In [1054]:
pd.set_option('display.max_rows', 80000)
df[df.isin([np.inf])].any()

id                                        False
marital_status                            False
vintage                                   False
claim_amount                              False
cltv                                      False
Age                                       False
Education                                 False
Occupation                                False
Income_velocity                           False
Num_claims                                False
Claim_amount_per_policy                   False
Claims_ratio                              False
Loyalty_score                             False
Ratio_claim_amount_to_income               True
Ratio_num_policies_to_vintage              True
Probability_claim_amount_given_income     False
Probability_num_policies_given_vintage    False
age                                       False
family_size                               False
no_of_claims                              False
spending_power                          

In [53]:
df[df.isin([np.inf, -np.inf]).any(1)]

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv,income_int,...,claim_num_policies,Ratio_vintage_to_policy,Frequency_gender,Frequency_area,Frequency_qualification,Frequency_income,Frequency_marital_status,Frequency_vintage,Frequency_claim_amount,Frequency_num_policies,Frequency_type_of_policy,Frequency_cltv,Probability_vintage_given_policy


In [912]:
df = df[df.isin([np.inf, -np.inf]).any(1)].replace(np.inf, np.nan)
df.fillna(0, inplace=True)

In [914]:
num_cols = [col for col in df.columns if df[col].dtypes != 'object']
for col in num_cols:
    scaler = RobustScaler()
    df.loc[:, col] = scaler.fit_transform(df[col].values.reshape(-1, 1))

In [55]:
X = df.drop('cltv', axis=1)
y = df['cltv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

ValueError: could not convert string to float: 'Male'

In [390]:
las_reg = Lasso()
las_reg.fit(X_train, y_train)
y_pred = las_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.0012597151908262383


In [391]:
rid_reg = Ridge()
rid_reg.fit(X_train, y_train)
y_pred = rid_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15498130562793977


In [392]:
gb_reg = GradientBoostingRegressor()
gb_reg.fit(X_train, y_train)
y_pred = gb_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15904743689419387


In [393]:
xgb_reg = xgb.XGBRegressor(eta=0.1)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.1541919677006145


In [394]:
lgm_reg = lgm.LGBMRegressor()
lgm_reg.fit(X_train, y_train)
y_pred = lgm_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15341277229488737


In [271]:
lin_reg = LinearRegression()
pipe_lr = Pipeline([('scaler', MinMaxScaler()), ('model', lin_reg)])
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.1547305263343366


In [272]:
lin_reg = Lasso()
pipe_lr = Pipeline([('scaler', StandardScaler()), ('model', lin_reg)])
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

-0.00018297917645515582


In [273]:
lin_reg = Ridge()
pipe_lr = Pipeline([('scaler', StandardScaler()), ('model', lin_reg)])
pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15504266079748918


In [214]:
ab_reg = AdaBoostRegressor()
ab_reg.fit(X_train, y_train)
y_pred = ab_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

-0.36405217173484283


In [215]:
dt_reg = DecisionTreeRegressor()
dt_reg.fit(X_train, y_train)
y_pred = dt_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

-0.8097699431620928


In [216]:
xgb_reg = xgb.XGBRegressor(eta=0.1)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15482667788719884


In [217]:
lgm_reg = lgm.LGBMRegressor()
lgm_reg.fit(X_train, y_train)
y_pred = lgm_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15591377820088914


In [57]:
perm = PermutationImportance(lin_reg, random_state=42).fit(X_train, y_train)
eli5.show_weights(perm, feature_names=X.columns.tolist())

ValueError: could not convert string to float: 'Male'

In [114]:
perm = PermutationImportance(ab_reg, random_state=42).fit(X_train, y_train)
eli5.show_weights(perm, feature_names=X_train.columns.tolist())

NameError: name 'ab_reg' is not defined

In [115]:
perm = PermutationImportance(dt_reg, random_state=42).fit(X_train, y_train)
eli5.show_weights(perm, feature_names=X_train.columns.tolist())

NameError: name 'dt_reg' is not defined

In [119]:
perm = PermutationImportance(xgb_reg, random_state=42).fit(X_train, y_train)
eli5.show_weights(perm, feature_names=X_train.columns.tolist())

Weight,Feature
0.2448 ± 0.0025,num_policies
0.0755 ± 0.0021,claim_num_policies
0.0747 ± 0.0012,claim_amount
0.0343 ± 0.0018,claim_amt_to_income_ratio
0.0258 ± 0.0011,id
0.0168 ± 0.0005,avg_claim_per_id
0.0146 ± 0.0011,policy
0.0115 ± 0.0012,marital_status
0.0100 ± 0.0008,area
0.0071 ± 0.0007,qualification


In [118]:
perm = PermutationImportance(lgm_reg, random_state=42).fit(X_train, y_train)
eli5.show_weights(perm, feature_names=X_train.columns.tolist())

Weight,Feature
0.2201 ± 0.0026,num_policies
0.0409 ± 0.0010,claim_num_policies
0.0289 ± 0.0008,claim_amount
0.0268 ± 0.0011,id
0.0235 ± 0.0003,avg_claim_per_id
0.0174 ± 0.0009,policy
0.0142 ± 0.0008,marital_status
0.0119 ± 0.0005,claim_amt_to_income_ratio
0.0092 ± 0.0010,area
0.0090 ± 0.0005,vintage


In [120]:
explainer = shap.TreeExplainer(lgm_reg)
shap_values = explainer.shap_values(y_pred)

ValueError: Input numpy.ndarray or list must be 2 dimensional

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values, y_pred)

In [ ]:
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test)

In [ ]:
k_explainer = shap.KernelExplainer(lin_reg.predict, X_train)
shap_values = explainer.shap_values(y_pred)
shap.force_plot(k_explainer.expected_value, shap_values, y_pred)

In [29]:
df['premium_vintage'] = df['premium'] / df['vintage']
df['claim_rate'] = df['claim_amount'] / df['premium']
df['policy_coverage'] = df.groupby('id')['coverage'].transform('sum')
df['policy_diversity'] = df.groupby('id')['type_of_policy'].transform('nunique')
df['claim_history'] = df.groupby('id')['claim_amount'].transform(lambda x: (x > 0).any())
df['age'] = pd.to_datetime('today').year - df['dob'].dt.year
df = pd.concat([df, pd.get_dummies(df['gender'], prefix='gender')], axis=1)
df = pd.concat([df, pd.get_dummies(df['marital_status'], prefix='marital_status')], axis=1)
df['income_bracket'] = pd.cut(df['income'], bins=[0, 200000, 500000, 1000000, 5000000], labels=['low', 'medium', 'high', 'very high'])
df = pd.concat([df, pd.get_dummies(df['income_bracket'], prefix='income_bracket')], axis=1)
df['premium_category'] = pd.cut(df['premium'], bins=[0, 50000, 100000, 200000, 5000000], labels=['low', 'medium', 'high', 'very high'])
df = pd.concat([df, pd.get_dummies(df['premium_category'], prefix='premium_category')], axis=1)

KeyError: 'premium'

In [ ]:
Premium per vintage: The average premium paid by a customer per year of policy tenure.

Claim rate: The ratio of the total amount claimed by a customer to the total premium paid.

Policy coverage: The total sum insured for all policies held by a customer.

Policy diversity: The number of different types of policies held by a customer.

Claim history: A binary feature indicating whether a customer has made a claim in the past.

Age: The age of the customer at the time of policy purchase.

Gender dummies: Create dummies for gender, this will capture the effect of gender on CLTV

Marital_status dummies: Create dummies for marital status, this will capture the effect of marital status on CLTV

Income_bracket: Create a categorical feature based on the income bracket of the customer

Premium_category: Create a categorical feature based on the premium bracket of the customer

In [ ]:
Policy tenure: The number of years that a customer has held an active policy with the company.

Average claim amount: The average amount claimed by a customer across all policies.

Policy mix: The ratio of different types of policies held by a customer.

Income per policy: Divide total income of customer by number of policies held.

Vintage per policy: Divide vintage by number of policies held.

Claim per policy: Divide claim amount by number of policies held.

Premium per policy: The average premium paid by a customer per policy.

Policy retention rate: The percentage of policies that a customer renews over time.

Claim frequency: The number of claims made by a customer per year.

Area Dummies: Create dummies for area, this will capture area wise variation in CLTV

Interaction features: Create interaction features between different numerical features, this will capture the combined effect of different features.

Policy dummies: Create dummies for policy type, this will capture the effect of different policy on CLTV.

Qualification dummies: Create dummies for customer's highest qualification, this will capture the effect of education level on CLTV